# Animating a 3-D Robot: Inverse Kinematics and Cost-Function Minimization



In [2]:
# multimodal function
import numpy as np
import matplotlib.pyplot as plt

---
This notebook implements version of the method described in the following paper: 
- David E. Breen. Cost minimization for animated geometric models in computer graphics. The Journal of Visualization and Computer Animation, 8(4):201–220, 1997.
---

![](figs/robot_path.png)
**Figure 1:** Goal-oriented animation: The robot arm moves from one target location to another while avoiding obstacles. The yelllow spheres represent the targets. 

## The cost function 

The animation is achieved by using the gradient-descent algorithm to solve the following minimization problem: 

$$
\begin{align}
	\hat{\bf \Phi} = \arg_{{{\bf \Phi}}}\min  C\left({\bf \Phi}\right),
	\label{simple_motion_min}
\end{align}
$$

where the cost function $C$ is defined by:

$$
\begin{align}
	C\left({\bf \Phi}\right) = \underbrace{\|{\bf e}\left({\bf \Phi}\right) - {\bf g}\|}_{\text{goal attraction}} + \underbrace{\sum_{i=1}^{n} \mathcal{F}_R\left(\|{\bf e}\left({\bf \Phi}\right) - {\bf o}_i\|\right)}_{\text{obstacle-avoidance penalty}} + \underbrace{\sum_{j=1}^{m} \mathcal{L}\left(\phi_j\right)}_{\text{Joint-range limit}}.
	\label{simple_motion}
\end{align}
$$

Here, ${\bf g} = \left(x_g, y_g, z_g\right)^\mathsf{T}$ is the *goal location*, ${\bf o}_i$ is the location of obstacle $i$. The summation limits are $n$, which is the numeber obstacles, and $m$, which is the number of joints to be constrained. Function ${\bf e}\left({\bf \Phi}\right)$ computes the arm's *forward kinematics* and returns the location of the arm's tip ${\bf e} = \left( e_x, e_y, e_z \right)^\mathsf{T}$, i.e., the *end-effector's location,* given the arm's joint angles, ${\bf \Phi} = \left( \phi_1, \phi_2,  \phi_3, \phi_4\right)^\mathsf{T}$.   Function $\mathcal{F}_R$ is a collision-avoidance penalty field. It penalizes poses that take the end effector too close to an obstacle, i.e., beyond a pre-defined distance $R$. The third component of Equation $\ref{simple_motion}$  limits the range of each joint angle. Function $\mathcal{L}$ is another penalty function. Its value increases as the joint angle $\phi_j$ approaches its maximum or minimum limit. Outside these limits, $\mathcal{L}$ vanishes. 

Next, we describe the components of the cost function in  ($\ref{simple_motion}$) in more detail. 


### The attraction term

The goal-attraction term measures the distance between the end effector's current position and the goal location. During the animation, the attraction term guides the end-effector towards the goal location. Since its equation is the Euclidean norm (or Euclidean distance), the minimal-solution path, when disregarding obstacles and joint range limits, is a straight line from the end-effector's current location to the goal location. 

$$
\begin{align}
	C_1\left({\bf \Phi}\right) = \|{\bf e}\left({\bf \Phi}\right) - {\bf g}\|
\end{align}
$$


In [3]:
def goal_attraction(e_Phi: np.ndarray, g: np.ndarray) -> float:
    """Computes the value of the goal-attraction cost 
    
    Args:
        e_Phi (np.ndarray): A 3x1 column matrix containing the 
                        (ex,ey,ez) coordinates of the current 
                        location of the end effector.
                        
        g (np.ndarray): A 3x1 column matrix containing the 
                        (gx,gy,gz) coordinates of the 
                        location of the goal point.

    Returns:
        cost (float):   The calculated cost 
    
      """
    
    c = np.linalg.norm(e_Phi - g)
    return c    # float 

### The field potential function for obstacle avoidance

The field potential is a *penalty term* that increases its value as the end effector approaches an obstacle. It is defined as follows: 

$$
\begin{align}
        \mathcal{F}_R\left(d\right) = 
	\begin{cases}
		\ln\left(R/d\right), & 0 < d \leq R,\\
		0,                         &d > R.
	\end{cases}
	\label{field_function1}
\end{align}	
$$

In [5]:
def F_R(d: float) -> float:
    """Computes the value of the field potential for 
       obstacle-avoidance cost 
    
    Args:
        d (np.float): Distance between the end-effector and 
                      the obstacle.                        

    Returns:
        cost (float):   The calculated cost 
    
      """
    
    # Assume that all objects have the same size. 
    # Radius (approx) of the object. It might 
    # need to be a bit larger than the actual 
    # radius of the object.
    R = 50
    cost = 2;
    if (d <= R) and (d > 0):
        cost = np.log(R/d)
    elif (d > R):
        cost = 0  
    
    return cost   # float 

In [1]:
# End effector position
#e_Phi = np.array([[3],
#                  [0],
#                  [0]])

# Obstacle position
#obstacle = np.array([[10, 10],
#                     [15, 0],
#                     [20, 0]])

#print (obstacle[:,1])
# Distance between end effector and obstacle
#i = 0
#d = np.linalg.norm(e_Phi - obstacle[:,i])
#print(d)

# Cost of obstacle avoidance
#print("Cost of obstacle (", i, ") = ", F_R(d))

### The "range-of-motion" limit function  

The limit function constrains the range of motion of the joints (i.e., angles), and is given by: 

$$
\begin{align}
        \mathcal{L}\left(\phi\right) = 
	\begin{cases}
		\ln\left(\delta/\left(\phi-\phi_\text{min}\right)\right), & \phi_\text{min} < \phi \leq \phi_\text{min} + \delta\\
		0,                         & \phi_\text{min} + \delta < \phi < \phi_\text{max} - \delta\\
		\ln\left(\delta/\left(\phi_\text{max} - \phi\right)\right), & \phi_\text{max} - \delta \leq \phi < \phi_\text{max},\\
	\end{cases}
	\label{limit_function}
\end{align}	
$$

where $\phi$ is the joint angle, $\phi_\text{min}$ and $\phi_\text{max}$ are the limits of that joint, and $\delta$ is the angular distance from each of the limits after which the limit function vanishes. 

In [7]:
def L(phi:float, min_:int, max_:int, delta_:int) -> float:

    # Assume all limits and delta to be the same
    phi_min = min_        # Joint's minimum angle limit.                        
    
    phi_max = max_       # Joint's maximum angle limit.
    
    delta   = delta_       # The angular distance from each of 
                       # the limits after which the limit 
                       # function vanishes.
    
    print(phi)
    
    # Temp value to return 
    cost = 2
    
    # Cost function
    if (phi > phi_min) and (phi <= (phi_min+delta)):
        cost = np.log(phi / (phi-phi_min))
    elif (phi > phi_min + delta) and (phi < phi_max - delta):
        cost = 0
    elif (phi >= phi_max - delta) and (phi < phi_max):
        cost = np.log(phi / (phi_max - phi))
    
    return cost   # float 

In [8]:
def end_effector(Phi):
    theta_arr = Phi
    
    # Arm base x,y,z coordinates / translations
    arm_x = 5
    arm_y = 5
    arm_z = 0
    
    # Length of each arm component extending from join
    length_arr = [2, 5, 5, 5] 
    
    # Local transformation matrices 
    
    # Frame 1 written w.r.t. Frame 1 (global)
    T_01 = np.array([[np.cos(theta_arr[0]), -np.sin(theta_arr[0]), 0.0, arm_x],
                    [np.sin(theta_arr[0]), np.cos(theta_arr[0]) , 0.0, arm_y],
                    [0.0                 , 0.0                  , 1.0, arm_z],
                    [0.0                 , 0.0                  , 0.0, 1.0]]
                   )
    # Frame 1 written w.r.t. Frame 2
    T_12 = np.array([[np.cos(theta_arr[1]) , 0.0, np.sin(theta_arr[1]), 0.0],
                    [0.0                  , 1.0, 0.0                 , 0.0],
                    [-np.sin(theta_arr[1]), 0.0, np.cos(theta_arr[1]), length_arr[0]],
                    [0.0                  , 0.0, 0.0                 , 1.0]]
                   )
    # Frame 3 written w.r.t. Frame 2
    T_23 = np.array([[np.cos(theta_arr[2]) , 0.0, np.sin(theta_arr[2]), 0.0],
                    [0.0                  , 1.0, 0.0                 , 0.0],
                    [-np.sin(theta_arr[2]), 0.0, np.cos(theta_arr[2]), length_arr[1]],
                    [0.0                  , 0.0, 0.0                 , 1.0]]
                   )
    # Frame 4 written w.r.t. Frame 3
    T_34 = np.array([[np.cos(theta_arr[3]) , 0.0, np.sin(theta_arr[3]), 0.0],
                    [0.0                  , 1.0, 0.0                 , 0.0],
                    [-np.sin(theta_arr[3]), 0.0, np.cos(theta_arr[3]), length_arr[2]],
                    [0.0                  , 0.0, 0.0                 , 1.0]]
                   ) 
    
    # Local-to-global transformation matrices 
    T_04 = T_01 @ T_12 @ T_23 @ T_34                    # Frame 4 written w.r.t. Frame 0
    e = T_04[0:3, 3]
    
    return e 

In [2]:
#phi = np.array([1, 2, 3, 4])
#print(end_effector(phi))

In [10]:
# Cost function 
def C(Phi, goal_location, obstacle_locations, angle_min, angle_max, angle_delta): 
    
    # Goal (target) location 
    g = goal_location

    # End-effector location 
    e_Phi = end_effector(Phi)

    # Location of obstacles 
    obstacle = obstacle_locations # A 2d array
    num_obstacles = obstacle.size # Number of rows of 2d array

    cost = 0

    # Goal attraction
    cost = np.linalg.norm(e_Phi - g) 

    # Obstacle avoidance penalty
    cost += sum(F_R(np.linalg.norm(e_Phi - obstacle[:,j])) for j in range(num_obstacles))

    # Joint-range limit
    cost += sum(L(phi, angle_min, angle_max, angle_delta) for phi in Phi)  
    return cost

### Numerical approximation of the Jacobian matrix 

The first column vector of this Jacobian describes how the end-effector position changes for small changes in joint angle $\phi_1$. Similarly, the second column vector describes how the end-effector position changes for small changes in joint angle $\phi_2$. Note that each component of the Jacobian matrix in Equation $\ref{jacobian_e}$ is equivalent to a 1st-order derivative of a scalar function of a single variable, and can be approximated numerically using finite differences. The approximated Jacobian is then: 

$$
\begin{align}
          J\left({\bf e},{\bf \Phi}\right) \approx  
          \begin{bmatrix}
          	 \dfrac{\Delta{\bf e}}{\Delta\phi_1}  & \dfrac{\Delta{\bf e}}{\Delta\phi_2}  & \dfrac{\Delta{\bf e}}{\Delta\phi_3}  & \dfrac{\Delta{\bf e}}{\Delta\phi_4}
          \end{bmatrix} =           
          \begin{bmatrix}
          	  \dfrac{\Delta e_x}{\Delta\phi_1}  & \dfrac{\Delta e_x}{\Delta\phi_2}   & \dfrac{\Delta e_x}{\Delta\phi_3}  & \dfrac{\Delta e_x}{\Delta\phi_4}\\[1em]
          	 \dfrac{\Delta e_y}{\Delta\phi_1}  & \dfrac{\Delta e_y}{\Delta\phi_2}   & \dfrac{\Delta e_y}{\Delta\phi_3}  & \dfrac{\Delta e_y}{\Delta\phi_4}\\[1em]	 
          	 \dfrac{\Delta e_z}{\Delta\phi_1}  & \dfrac{\Delta e_z}{\Delta\phi_2}   & \dfrac{\Delta e_z}{\Delta\phi_3}  & \dfrac{\Delta e_z}{\Delta\phi_4}	 
	 \end{bmatrix}.          
	\label{jacobian_e_approx}
\end{align}
$$

for a small $\Delta \phi_i, i=1,\dots,4$, where the elements of the matrix in ($\ref{jacobian_e_approx}$) are forward finite differences. We can also write the columns of the Jacobian matrix in terms of ${\bf e}\left(\Phi\right)$ as follows:

$$
\begin{align}
        \frac{\Delta {\bf e}}{\Delta \phi_i} = \frac{{\bf e}\left({\bf \Phi}+\Delta \phi_i\right) - {\bf e}\left({\bf \Phi}\right)}{\Delta \phi_i},
	\label{approxExample}
\end{align}	
$$

for small $\Delta \phi_i$, and $i=1,\dots,4$. Note that the displacement of ${\bf \Phi}$ by $\Delta \phi_i$ in the calculation of ${\bf e}\left({\bf \Phi}+\Delta \phi_i\right)$ in Equation $\ref{approxExample}$ is applied only to the corresponding joint angle $\phi_i$ which is a component of ${\bf \Phi}$, and not to all components. For example, the position of the end-effector displaced by $\Delta \phi_1$ is:

$$
\begin{align}
       {\bf e}\left({\bf \Phi}+\Delta \phi_1\right) = {\bf e}\left(
       \begin{bmatrix}
       		\phi_1\\   \phi_2\\ \phi_3\\   \phi_4
       \end{bmatrix}
      +
       \begin{bmatrix}
       		\Delta \phi_1\\   0\\ 0\\   0
       \end{bmatrix}      
      \right).
	\label{incrementPhi}
\end{align}
$$

The other displacements are calculated analogously. 

In [3]:
def J(end_effector_function, Phi):

    e = end_effector_function
    
    # Delta change in each angle
    deltas = [0.1, 0.1, 0.1, 0.1]
    
    # Delta change in each angle in matrix form, for easier addition
    deltas_matrix = np.array([
                      [0.1, 0, 0, 0],
                      [0, 0.1, 0, 0],
                      [0, 0, 0.1, 0],
                      [0, 0, 0, 0.1]])
    
    # Initilize Jacobian matrix
    J = np.empty((3, 1))
    
    # For each angle
    for i in range(4): 
        
        # Calculate end-effector delta
        e_delta = ( e(Phi + deltas_matrix[i]) - e(Phi))
        
        # Calculate numerical estimation of partial derivative,
        DxDphi = e_delta[0] / deltas[i]
        DyDphi = e_delta[1] / delta[i]
        DzDphi = e_delta[2] / delta[i]
        
        # Append resulting values to jacobian matrix
        J = np.concatenate( (J, np.array([[DxDphi], [DyDphi], [DzDphi]])), axis=1)
    
    """Computes the value of the function at x
    
    Args:
        end_effector_function: handle to end_effector function 
 
    Returns:
        Jacobian (np.ndarray): A 3x4 Jacobian matrix
    """

    # Filter out extra columns/rows and return
    return J[:,1:5]


In [3]:
#J (end_effector, phi)

### The gradient-descent solution for inverse-kinematics

![ik_algorithm](figs/ik_algorithm.png)

In [16]:
def gradient_descent_IK(pos, target_pos): 
    """Solves the inverse-kinematics problem using the gradient descent
    """

    delta_f = target_pos - f(pos)           # Difference between predicted and target
    dist = np.linalg.norm(delta_f)          # Distance from target

    Jac = J(end_effector, Phi)              # Compute the Jacobian for the current pose
    Jinv = np.linalg.inv(J)                 # Invert the Jacobian matrix
    delta_pos_mapped = Jinv @ delta_f       # Compute the change in pose
    f_predicted = f(pos + delta_pos_mapped) # Apply change to joint angles
    
    return uv_g

In [ ]:
xtrace = x                      # Stores the trajectory in x-domain
delta_f = uv_g - f(x)           # Difference between predicted and target 
dist = np.linalg.norm(delta_f)  # Error measure (distance)
table = []                      # Table to store the iteration results                        

it = 0                          # Iteration count 

# Delta f 
while dist > 0.1:    
    delta_f = uv_g - f(x)
    dist = np.linalg.norm(delta_f)

    # Jacobian at x
#    J = Jacobian(x)                       # Analytical Jacobian when available

    # Delta x for the Delta f using the inverse Jacobian
    delta_x_mapped = Jinv @ delta_f

    # Predicted function f(x + delta_x)
    f_predicted = f(x + delta_x_mapped)

    # The goal function value we want to achieve
    uv_g = f(xg)

In [2]:
# Cost minimization
# Seems similar to calculating Jacobian
def calculate_gradient(PhiArr, goal, obstacles, ang_max, ang_min, lim_delta):
    
    # Delta change in each angle/phi
    deltas = [0.1, 0.1, 0.1, 0.1]
    
    # Delta change in each angle in matrix form, for easier addition
    deltas_matrix = np.array([[0.1, 0, 0, 0],
                              [0, 0.1, 0, 0],
                              [0, 0, 0.1, 0],
                              [0, 0, 0, 0.1]])
    
    # Empty gradient
    gradient = np.empty((1, 1))
    
    # For each angle in arm
    for i in range(4) :
        
        # C(Φ + ΔΦₙ) 
        cost_with_PhiDelta = C(PhiArr + deltas_matrix[i], goal, obstacles, ang_max, ang_min, lim_delta)
        # C(Φ)
        cost_without_PhiDelta = C(PhiArr,  goal, obstacles, ang_max, ang_min, lim_delta)
        
        # Calculate components of gradient
        # ( C(Φ + ΔΦₙ) -  C(Φ) )/ ΔΦₙ
        deltaC = (cost_with_Phidelta - cost_without_Phidelta) / deltas[i]
        
        # Add component to gradient matrix
        gradient.append(deltaC)

    return gradient

In [3]:
def animate_robot(p, e, goal):
    
    # Obstacle locations. Change these to whatever you want
    obstacles = np.array([[0, 0, 0],
                          [0, 0, 0],
                          [0, 0, 0]])
    
    # Angle limits. Change these to whatever you want
    # Set to full range of motion by default
    ang_max = 360
    ang_min = 0
    
    ang_delta = 10
    
    # Steps 1, 2, 3, 4, and 5 respectively
    PhiArr = p.transpose()
    end_effector = e.transpose()
    goal_pos = goal.transpose()
    distance = 0.01
    gradient_step = 0.1
    
    while (np.abs(end_effector - goal_pos) > distance):
        # Calculate new joint-angle (step 7)
        # Also just replace the old config with the new one (step 10)
        PhiArr = PhiArr - gradient_step * calculate_gradient(PhiArr, goal_pos, obstacles, ang_max, ang_min, ang_delta)
        
        # Draw robot arm (step 8)
        drawRobotArm(PhiArr)
        
        # Calculate new end-effector location, update (step 9)
        end_effector = end_effector(PhiArr)